# JGBYieldCurvePCA.ipynb

## 財務省のホームページから日本国債金利のcsvファイルを取得、jgbcm_all.csv と jgbcm.csvをマージして過去10年の履歴を抽出して。結果はcsvファイルに格納して。財務省のファイルコードはshift JIS、1行目はスキップして。日付は和暦なので日付変換をお願い。

財務省サイトから jgbcm_all.csv / jgbcm.csv を取りにいって、マージ＋直近10年だけ切り出して CSV に保存する Jupyter Notebook 用コードを書きました。

そのままコピペして実行すれば OK です。

取得元
- 過去データ: https://www.mof.go.jp/jgbs/reference/interest_rate/data/jgbcm_all.csv 
- 最新データ: https://www.mof.go.jp/jgbs/reference/interest_rate/jgbcm.csv

In [ ]:
import pandas as pd
import numpy as np
import re

# -------------------------------
# ① 財務省データ取得
# -------------------------------
url_all = "https://www.mof.go.jp/jgbs/reference/interest_rate/data/jgbcm_all.csv"
url_recent = "https://www.mof.go.jp/jgbs/reference/interest_rate/jgbcm.csv"

df = pd.concat([
    pd.read_csv(url_all, skiprows=1, encoding="shift_jis"),
    pd.read_csv(url_recent, skiprows=1, encoding="shift_jis")
])

# -------------------------------
# ② 和暦 → 西暦変換
# -------------------------------
def convert_wareki(x):
    if not isinstance(x, str):
        return None
    m = re.match(r"([RHS])(\d+)\.(\d+)\.(\d+)", x)
    if not m:
        return None
    e, y, mo, d = m.groups()
    base = {"R":2018, "H":1988, "S":1925}[e]  # R1=2019, H1=1989, S1=1926方式
    return f"{base + int(y)}-{mo}-{d}"

df["date"] = pd.to_datetime(df["基準日"].apply(convert_wareki))
df = df.set_index("date").drop(columns=["基準日"])

# 数値変換
df = df.replace("-", np.nan).apply(lambda c: pd.to_numeric(c, errors="coerce"))

# -------------------------------
# ③ 過去10年に絞る
# -------------------------------
end = df.index.max()
start = end - pd.DateOffset(years=10)
df10 = df[start:end]

# -------------------------------
# ④ ローカル保存
# -------------------------------
filename = "jgb_yield_history.csv"
df10.to_csv(filename, encoding="utf-8")

print(f"📁 保存完了 → {filename}")
print(f"データ範囲: {df10.index.min().date()} 〜 {df10.index.max().date()}")
print(f"列数（年限）：{len(df10.columns)}")


## 取得したcsvから金利のイールドカーブを描いて。横軸に満期、縦軸に金利をプロット。日付を指定しその日のカーブをグラフ化、日付をスライドバーで動かし、カーブの動きが見えるようにして。縦軸の目盛りは固定、最大・最小が入るように調整、マイナス金利領域はグレーで。

了解です。
ここまで取得した 日本国債の日次利回りデータを使い、
イールドカーブをスライダー付きで動かせる Jupyter Notebook 完成版を作ります。

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# -------------------------------
# ① CSV読み込み
# -------------------------------
df10 = pd.read_csv("jgb_yield_history.csv", index_col=0, parse_dates=True)

# -------------------------------
# ② 年限の順序
# -------------------------------
labels = ["1年","2年","3年","4年","5年","6年","7年","8年","9年","10年",
          "15年","20年","25年","30年","40年"]

available = [l for l in labels if l in df10.columns]
years = np.array([int(x.replace("年","")) for x in available])
dates = df10.index.tolist()

# -------------------------------
# ③ y軸固定レンジ
# -------------------------------
y_min = float(np.nanmin(df10[available].values))
y_max = float(np.nanmax(df10[available].values))


# -------------------------------
# ④ Plotly Figure 初期生成
# -------------------------------
fig = go.Figure()

for i, d in enumerate(dates):
    fig.add_trace(go.Scatter(
        x=years,
        y=df10.loc[d, available],
        mode="lines+markers",
        name=str(d.date()),
        visible=(i == len(dates)-1)  # ← 最後の日付だけ表示
    ))

# -------------------------------
# ⑤ スライダー設定
# -------------------------------
steps = []
for i, d in enumerate(dates):
    steps.append(dict(
        method="update",
        args=[
            {"visible": [j == i for j in range(len(dates))]},
            {"title": {"text": f"日本国債イールドカーブ: {d.strftime('%Y-%m-%d')}",
                       "x": 0.5, "xanchor": "center"}}
        ]
    ))

# -------------------------------
# ⑥ レイアウト更新（タイトルは設定しない＝スライダー管理）
# -------------------------------
fig.update_layout(
    sliders=[{
        "active": len(dates)-1,
        "steps": steps
    }],
    xaxis_title="満期 (年)",
    yaxis_title="利回り (%)",
    template="plotly_white",
    yaxis=dict(range=[y_min, y_max]),
    height=800
)

# -------------------------------
# ⑦ マイナス金利ゾーン影
# -------------------------------
fig.add_hrect(
    y0=min(0, y_min), y1=0,
    fillcolor="lightgray", opacity=0.45, line_width=0
)

# -------------------------------
# ⑧ 初期タイトルをスライダー最終値と同期
# -------------------------------
fig.update_layout(
    title={
        "text": f"日本国債イールドカーブ: {dates[-1].strftime('%Y-%m-%d')}",
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()


🎉 実行するとできること

- スライダーを動かす → カーブがリアルタイムで動く
- 通常期・ゼロ金利・マイナス金利時代の形状変化が視覚化
- シャドウで 負金利期間が一目で分かる

次の発展ステップ（選択）
オプション	内容
- アニメーション：自動再生ボタン追加	
- 2年-10年スプレッド分析（景気先行シグナル）	
- PCA（金利因子：level/slope/curvature分解）	

## PCA分析でレベル・スロープ・カーブを可視化して。

了解です。
では 「PCA分析による金利構造可視化」 を追加します。

目的は👇

日本国債イールドカーブがどの因子（Level / Slope / Curvature）で動いているか分析し、
その因子動きを可視化し、さらにスライダー付きでカーブ再生成できるようにする。

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import plotly.graph_objects as go

# -------------------------------
# ① データ読み込み
# -------------------------------
df10 = pd.read_csv("jgb_yield_history.csv", index_col=0, parse_dates=True)

# -------------------------------
# ② 対象年限
# -------------------------------
labels = ["1年","2年","3年","4年","5年","6年","7年","8年","9年","10年",
          "15年","20年","25年","30年","40年"]

available = [l for l in labels if l in df10.columns]
years = np.array([int(x.replace("年","")) for x in available])

# -------------------------------
# ③ 欠損処理（推奨形式）
# -------------------------------
df_filled = df10[available].ffill().bfill()

# -------------------------------
# ④ PCA（主成分=3）
# -------------------------------
pca = PCA(n_components=3)
scores = pca.fit_transform(df_filled.values)
loadings = pca.components_
explained = pca.explained_variance_ratio_

print("📌 PCA寄与率（説明力）")
for i, r in enumerate(explained):
    print(f" PC{i+1}: {r*100:.2f}%")


# -------------------------------
# ⑤ PCA Loading Plot
# -------------------------------
fig = go.Figure()

for i in range(3):
    fig.add_trace(go.Scatter(
        x=years,
        y=loadings[i],
        mode="lines+markers",
        name=f"PC{i+1} ({explained[i]*100:.1f}%)"
    ))

fig.update_layout(
    title="📈 PCA因子形状 | PC1:Level / PC2:Slope / PC3:Curvature",
    xaxis_title="満期 (年)",
    yaxis_title="成分値",
    template="plotly_white",
    height=500
)

fig.show()


出力結果（期待される形）

- PC1（レベル） → すべての年限がほぼ同方向 →「全体的な金利水準の上下」
- PC2（スロープ） → 短期と長期で符号逆転 →「傾き（スティープ化/フラット化）」
- PC3（カーブ） → 中短期が上、長期が下 or 逆 →「山型・逆山型カーブ」

👉 これにより、日本国債利回りの変動理由を要因別に理解できます。

次のセルはPCAスライダー付き「金利再構築可視化」

「レベル・傾き・カーブの寄与を変えるとカーブがどう変わるか」

👇これが視覚化スライダー版です：

In [ ]:
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np

# -------------------------------
# ① ベースカーブ（平均）
# -------------------------------
base_curve = df_filled.mean().values

# -------------------------------
# ② UI部品
# -------------------------------
pc_selector = widgets.Dropdown(
    options=[("PC1 (Level)", 0), ("PC2 (Slope)", 1), ("PC3 (Curvature)", 2)],
    value=0,
    description="成分選択:",
)

date_slider = widgets.SelectionSlider(
    options=dates,
    value=dates[-1],
    description="📅 日付",
    continuous_update=True,
    style={"description_width": "100px"}
)

# -------------------------------
# ③ 描画関数
# -------------------------------
def update_plot(selected_pc, date):
    idx = dates.index(date)
    
    # 当日の主成分スコア
    pc1_val, pc2_val, pc3_val = scores[idx]
    
    # 対象PCのみ動かす
    active_pc_score = [pc1_val, pc2_val, pc3_val][selected_pc]

    # カーブ再構築（選択中PCのみ寄与）
    reconstructed = base_curve + active_pc_score * loadings[selected_pc]
    
    # 実測値
    actual = df_filled.loc[date, available].values

    fig = go.Figure()

    # ベースカーブ
    fig.add_trace(go.Scatter(
        x=years, y=base_curve,
        mode="lines",
        name="平均カーブ",
        line=dict(color="gray", dash="dot")
    ))

    # 寄与カーブ
    fig.add_trace(go.Scatter(
        x=years, y=reconstructed,
        mode="lines+markers",
        name=f"{pc_selector.label.replace(':','')}寄与カーブ",
        line=dict(width=3)
    ))

    # 実測カーブ
    fig.add_trace(go.Scatter(
        x=years, y=actual,
        mode="lines+markers",
        name=f"実測: {date.strftime('%Y-%m-%d')}",
        line=dict(color="black")
    ))

    fig.update_layout(
        title=f"📈 PCA影響可視化 | {pc_selector.options[selected_pc][0]} | {date.strftime('%Y-%m-%d')}",
        xaxis_title="満期 (年)",
        yaxis_title="利回り (%)",
        template="plotly_white",
        height=800,
        legend_title="系列",
    )

    fig.show()


# -------------------------------
# ④ UI連動
# -------------------------------
widgets.interact(update_plot, selected_pc=pc_selector, date=date_slider);


✔ これでできること
- 金利カーブの PCA 分解
- 各因子の形状プロット
-「Level/Slope/Curve」の理論確認
- スライダー操作でカーブ組み立て体験
- マクロイベント時のカーブ挙動理解
- 
📌 次追加できます：
- 1️⃣ 各日の PCA因子スコアをヒートマップ化	
- 2️⃣ PCA因子の時系列チャート作成	
- 3️⃣ PC1 (Level) と日銀政策金利の相関分析	
- 4️⃣ PC2スロープと景気サイクル分析	
- 5️⃣ 全てNotebookとして完成形に整理

## 平均カーブ、実測カーブ、それにPCカーブは何を意味している？

いい質問です。
この可視化は「金利カーブがどのような要因で動いているか」を分解して理解するためのものです。
それぞれの線の意味はまったく役割が違うため、ここで整理します👇

📌 3本の線の意味
|線|表示内容|役割・意味|
|----|----|----|
|① 平均カーブ (Base Curve / Mean Yield Curve)|過去10年の同じ年限の平均利回り|金利カーブの基準形状。そこから市場がどの方向に歪んでいるか比較する軸|
|② PC寄与カーブ (Principal Component Contribution Curve)|平均カーブ ＋ (その日の PCスコア × そのPCの形状)|特定の動き (Level / Slope / Curvature) が金利曲線をどう変形しているかを示す理論曲線|
|③ 実測カーブ (Actual Yield Curve)|財務省データから取得した実際の金利|実際の市場が反映されている形状。観測値|


🔍 図でイメージすると

                   実測カーブ（現実）
                         ▲
                         │
                     PC寄与カーブ（要因による動き）
                         ▲
                         │
                   平均カーブ（基準）


🧠 それぞれの意味をもう少し深く：

-----------
✔ ① 平均カーブ（Baseline / Neutral Shape）
- 過去10年の平均で作った「安定した基準曲線」
- これは何も変動が起きていない場合の金利カーブの理論的形状
-  👉 例えるなら 「平常時の姿勢」。
------------
✔ ②PCカーブ（そのPC成分の寄与のみ）
- これはPCAで抽出した変動要因を 単独で反映した場合の金利カーブ です。

|PC成分|意味|カーブの変形|
|---|---|---|
|PC1: Level（レベル）|金利の全体上昇/下落（政策金利・マクロ要因）|カーブが平行移動|
|PC2: Slope（スロープ）|2年と10年など、短期と長期の差（景気循環）|傾きが強調（スティープ・フラット）|
|PC3: Curvature（カーブ）|中年限（5〜10年）が膨らむ or 凹む（政策転換期・需給バタフライ）|「お椀型/くぼみ型」の形状化|


- 👉 PCカーブは市場動きの原因（Factor）だけを視覚化したもの。
- 👉 実際の動きがどの構造に支配されているかが理解できる。
-----------------
✔  ③実測カーブ
- 実際に市場で観測された国債利回り
- PCモデルが「どこまで市場を説明できているか？」を比較するために使う
- 👉 例えるなら 「現実の姿」。
-----------------


📈 3本を見る意味
|比較|得られる洞察|
|---|---|
|実測 vs 平均|「通常」と比べ今の金利は高いか？低いか？|
|PC寄与 vs 平均|市場の動きがどの力（要因）で説明できるか？|
|実測 vs PC寄与|残差 → 需給・歪み・政策要因の未説明部分|


🧭 例：動きの解釈シナリオ
|市場状況|見た目|解釈|
|---|---|---|
|PC1↑|全体上昇|金融引き締め, 物価上昇|
|PC2↑|長短金利差拡大|景気回復、正常イールド|
|PC2↓|フラット or 逆イールド|景気後退サイン|
|PC3↑|中期利回りだけ動く|日銀政策転換前の期待変動|

🎯結論まとめ
- 平均カーブ → 「基準」	
- PCカーブ → 「どの要因が曲線を形作っているかの理論モデル」	
- 実測カーブ → 「実際の市場の形」	

## 今度は平均カーブとPC寄与カーブの差をグラフ化して。縦軸は固定してそれぞれのPCごとに最大最小が入るように調整して

了解です。
目的は
「平均カーブから見て、そのPC成分がどれだけ金利を押し上げ／押し下げているか可視化したい」
ですね：

つまり：
- PC寄与カーブ − 平均カーブ = PC由来のゆがみ(Contribution)


これにより：
- PC1 → 全体押し上げ/押し下げ（レベル変動）
- PC2 → 長短金利差（スティープ化 or フラット化）
- PC3 → バタフライ型変動（曲率変化）

が視覚的に理解できるようになります。

In [ ]:
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np

# -------------------------------
# UI部品（同じ）
# -------------------------------
pc_selector2 = widgets.Dropdown(
    options=[("PC1 (Level)", 0), ("PC2 (Slope)", 1), ("PC3 (Curvature)", 2)],
    value=0,
    description="成分選択:"
)

date_slider2 = widgets.SelectionSlider(
    options=dates,
    value=dates[-1],
    description="📅 日付",
    continuous_update=True,
    style={"description_width": "100px"}
)

# -------------------------------
# ① PC別に deviation の min/max を事前計算
# -------------------------------

pc_ranges = {}

for pc in range(3):
    deviations = []
    for i in range(len(dates)):
        deviations.append(scores[i][pc] * loadings[pc])

    # flattenして範囲取得
    dev_all = np.concatenate([np.array(d) for d in deviations])
    
    pc_min = np.min(dev_all)
    pc_max = np.max(dev_all)

    # 少し余白をつける
    margin = (pc_max - pc_min) * 0.10
    pc_ranges[pc] = (pc_min - margin, pc_max + margin)

pc_ranges


In [ ]:
def update_deviation(selected_pc, date):
    idx = dates.index(date)

    # deviation = PC寄与量
    pc_val = scores[idx][selected_pc]
    deviation = pc_val * loadings[selected_pc]

    ymin, ymax = pc_ranges[selected_pc]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=years,
        y=deviation,
        mode="lines+markers",
        name=f"{pc_selector2.options[selected_pc][0]} 寄与量",
        line=dict(width=3)
    ))

    fig.add_hline(y=0, line_width=1, line_color="gray")

    fig.update_layout(
        title=f"📊 PCA成分寄与量（差分） | {pc_selector2.options[selected_pc][0]} | {date.strftime('%Y-%m-%d')}",
        xaxis_title="満期 (年)",
        yaxis_title="平均からの差 (利回り％)",
        template="plotly_white",
        height=600,
        
        # ★ PC別にスケール適用
        yaxis=dict(range=[ymin, ymax])
    )

    fig.show()


widgets.interact(update_deviation, selected_pc=pc_selector2, date=date_slider2);


解釈メモ（推奨）
- PC1（Level） → 幅が広い → 金融政策の影響が大きい
- PC2（Slope） → 振れ幅が中程度 → 景気サイクルに対応
- PC3（Curvature） → 変動は小さいが政策転換点で動きやすい

次のアップグレード提案（要望に応じて）
- 1️⃣ deviation（PC寄与）+ 実測との差＝未説明残差（Risk Premium）分析
- 2️⃣ deviation をヒートマップ化 → 相場 regime の変化抽出
- 3️⃣ PCスコアの時系列と景気指標(景気D.I, GDP, インフレ)を重ねる
- 4️⃣ PCA＋回帰で「金利モデル化」（因果推定）
- 5️⃣ Dash / StreamlitでJGB Yield Analytics App構築

👇どれに進めたいですか？